## LangChain with Simple Chain Pattern
This example is setting up a LangChain example that connects to a custom OpenAI-compatible large language model (LLM) endpoint using credentials from the environment, then prompts it to suggest a provide a 5 lines of overview of "VMware Tanzu Hub"

In [ ]:
# set the working directory for the project
%cd /home/vcap/app/cf-jupyterlab-samples

### Step 1: Import the dependencies

In [ ]:
import sys, os
import requests
import json
import httpx
import warnings
from langchain.prompts import ChatPromptTemplate
from langchain.chains import LLMChain
from langchain_openai import ChatOpenAI
from tanzu_utils import CFGenAIService
warnings.filterwarnings('ignore')

### Step 2: Set up the OpenAI API credentials

In [ ]:
# load your service details replace name with your Gen AI service.  Gen AI service is bound to the app
chat_service = CFGenAIService("tanzu-gpt-oss-120b")

# List available models
models = chat_service.list_models()
for m in models:
    print(f"- {m['name']} (capabilities: {', '.join(m['capabilities'])})")

# construct chat_credentials
chat_credentials = {
    "api_base": chat_service.api_base + "/openai/v1",
    "api_key": chat_service.api_key,
    "model_name": models[0]["name"]
}

### Step 3: Initialize the LLM

In [ ]:
# 2. HTTP client (optional but recommended for custom config)
httpx_client = httpx.Client(verify=False)  # verify=False if your endpoint needs --insecure

# 3. Initialize the LLM
llm = ChatOpenAI(
    temperature=0.9,
    model=chat_credentials["model_name"],   # model name from CF service
    base_url=chat_credentials["api_base"],  # OpenAI-compatible endpoint
    api_key=chat_credentials["api_key"],    # Bearer token
    http_client=httpx_client
)

### Step 4: Create a prompt template, create a chain, and run the chain

In [ ]:
# Create a prompt template
prompt = ChatPromptTemplate.from_template(
    """Provide a concise 5-line overview of {topic}.
    Focus on its purpose, key capabilities, core components,
    and how it integrates within the VMware Tanzu ecosystem.
    Write in a clear, professional tone suitable for a technical summary."""
)
# Create the chain
chain = LLMChain(llm=llm, prompt=prompt)

# Define the topic
topic = "VMware Tanzu Hub"

# Run the chain
response = chain.run(topic)
print(response)